# https://penseeartificielle.fr/methode-google-comprendre-sens-mots-word-embedding-python-gensim/

# Corpus 1

In [ ]:
documents = [
"le chat mange ses croquettes",
"le chien aime ses croquettes",
"le chat ronronne et mange"
]

# Vecteurs TF-IDF de tous les mots dans chaque document

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tfidf_vectorizer = TfidfVectorizer()
tfidf_vector = tfidf_vectorizer.fit_transform(documents)
 
pd.DataFrame(
    tfidf_vector.toarray(),
    columns=tfidf_vectorizer.get_feature_names(),
    index=[f"doc_{i+1}" for i in range(len(documents))]
)

# Distance (cosinus) vectorielle entre 2 documents

In [ ]:
from scipy.spatial.distance import cosine
def print_cosine(i_doc1,i_doc2):
    print('doc1:',documents[i_doc1])
    print('doc2:',documents[i_doc2])
    print(cosine(
        tfidf_vector.toarray()[i_doc1],
        tfidf_vector.toarray()[i_doc2]
    ))

In [ ]:
# docs proches
print_cosine(0,2)

In [ ]:
# docs éloignés
print_cosine(1,2)

# Corpus 2

In [ ]:
documents = [
"le chat mange ses croquettes",
"le chien dévore ses croquettes",
"le chat dévore son paté",
"jean va travailler",
"le chat mange son repas",
"jacque aime quand son chien mange"
]

# Calcul du contexte (bag of words) global d'un mot à partir de chaque document

In [ ]:
from collections import defaultdict
contexts = defaultdict(list)
bag_size = 1
for phrase in documents:
    words = phrase.split()
    print()
    print('phrase:',words)
    # foreach word of a phrase by index, adjust context of the word 
    for i, word in enumerate(words):
        # index candidates for context
        candidate = [i+j for j in range (0-bag_size,1+bag_size) if j != 0]
        # local words context 
        bag_of_words = [words[c] for c in candidate if 0 <= c < len(words)]
        print('  + ',i,word,' | local bow:',bag_of_words)
        # add local context to global context
        contexts[word] = contexts[word] + bag_of_words
    print()
    display('New global context:',contexts)
        

# Prepare TF-IDF of documents

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(documents)
print(vectorizer.get_feature_names())
print(documents[0])
print(vectorizer.transform([documents[0]],len(vectorizer.get_feature_names())))

# Calcul pour chaque mot, cacul du vecteur TF-IDF du contexte (vu comme un document)

Si l’hypothèse distributionnelle est exacte, ces vecteurs de contexte peuvent être considérés comme des vecteurs de sens.


In [ ]:
vectors = {}
n_features = 2
for k, v in contexts.items():
    str_ = ' '.join(v)
    print(k,' - context: ',v,' -sentence: ', str_)
    vectors[k] = vectorizer.transform([str_], n_features)[0]
    print(vectors[k])

# Calcul de la similarité entre un mot un un autre en calculant la distance entre leur vecteur respectif de contexte

In [ ]:
def most_similar(word):
    scores = []
    keys_ = [k for k in vectors.keys() if k != word]
    for k in keys_:
        # to avoid "ValueError: dimension mismatch" -> .toarray() create a dense array (not optimal!) 
        scores.append(
          1-cosine(
              vectors[word].toarray(),
              vectors[k].toarray()
          )
        )
    return pd.Series(
        scores,
        index=keys_
      ).sort_values(ascending=False)#.head(1)

In [ ]:
display(documents)
most_similar('chat')